In [ ]:
import sys
sys.path.insert(0, "/projects/academic/smuldoon/bengieru/simplicial_model")

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
from Neuronal_Cascades_base import Geometric_Brain_Network
import os
import pickle
import multiprocessing

In [ ]:
def inner_loop(thresholds, network, path,
               size, memory, rest,
               TIME, C, trials, K, L, model_type):
    
    delta_matrix_row = np.zeros(len(thresholds))
    comm_sizes = []
    distance_matrices = []
    
    for i, threshold in enumerate(thresholds):
        neurons= [Geometric_Brain_Network.neuron(k, memory = memory, rest = rest, threshold = threshold) for k in range(size)]
        
        network.get_neurons(neurons)
        D, Q = network.make_distance_matrix(TIME, C, trials, K, L, model_type = model_type)
    
        comm_sizes.append(Q)
    
        delta_min, delta_max = network.compute_persistence(D, spy = True)
        plt.savefig(path + 'pers_diag_thresh=%.2f'%(threshold) + '.pdf')
        
        delta_matrix_row[i] = delta_min
    
    network.display_comm_sizes(comm_sizes, list(thresholds), TIME, C, trials, memory, rest, threshold, K, L)
    plt.savefig(path + 'contagion_size_thresh=%d'%(nGD) + '.pdf')
    
    with open(path +  "comm_sizes_thresh=%d.pkl"%nGD, "wb") as fp:
        pickle.dump(comm_sizes, fp)
    
    return(delta_matrix_row)

In [ ]:
size = 1000
GD = 6
nGDs = [i for i in range(GD+1)]
topology = 'Ring'


TIME = 1000
trials = 1
C = 10000
K = 0
L = -1
mt = 'line_segment_model'

memory = TIME
rest = 0
thresholds = np.linspace(0,0.5,21)

base_path = '/projects/academic/smuldoon/bengieru/simplicial_model/Model_Variations/line_segment_model/n=%d/K=%.2f/'%(size, K)

os.makedirs(base_path, exist_ok = True)

In [ ]:
delta_matrix_min = np.zeros((len(thresholds),len(nGDs)))
inpts = []
for j,nGD in enumerate(nGDs):
    network= Geometric_Brain_Network.Geometric_Brain_Network(size, geometric_degree = GD, nongeometric_degree = nGD, manifold = topology)
    
    path = base_path + 'nGD=%d/'%nGD
    os.makedirs(path, exist_ok = True)
    
    one_set = (thresholds, network, path, size, memory, rest, TIME, C, trials, K, L, mt)
    inpts.append(one_set)

In [ ]:
with multiprocessing.Pool(processes = 8) as pool:
    result = pool.starmap(inner_loop, inpts)

In [ ]:
for j,nGD in enumerate(nGDs):
    delta_matrix_min[:,j] = result[j]
    
with open(base_path + "delta_matrix_mins_%.1f.pkl"%K, "wb") as fp:
    pickle.dump(delta_matrix_min, fp)

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (20,10))
normalize = Normalize(vmin=0, vmax=1)
im = ax.imshow(delta_matrix_min, 
               origin = 'lower', 
               interpolation='nearest', 
               aspect='auto',
               cmap = 'copper', 
               extent = [-.05, (nGDs[-1])/(GD)+0.05, thresholds[0]-0.05, thresholds[-1]+0.05])
fig.suptitle('Ring Topology on %d nodes. GD = %d,  T = %d, K = %.2f, L=%.2f, Trials = %d, C = %d, FULL MEMORY, RESTLESS'%(size, GD, TIME, K, L,trials, C), fontsize = 25)
ax.set_title('Delta Mins vs nGD/GD', fontsize = 20)
ax.set_xticks(np.array(nGDs)/(GD))
ax.set_yticks(thresholds)
ax.set_xlabel('$alpha$ = nGD/GD', fontsize = 15 )
ax.set_ylabel('Thresholds', fontsize = 15)
ax.tick_params(axis = 'both', labelsize = 12)
fig.colorbar(im)
plt.savefig(base_path + 'bifurcation_diag_min.pdf')